In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_emotion.csv
/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_sentiment_data.csv


In [2]:
import nltk 
import re
import pandas as pd
df1=pd.read_csv('/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_emotion.csv')
df2=pd.read_csv('/kaggle/input/sentiment-and-emotion-analysis-dataset/archive/combined_sentiment_data.csv')
df2.head()


,sentence,sentiment
0,So there is no way for me to plug it in here i...,negative
1,"Good case, Excellent value.",positive
2,Great for the jawbone.,positive
3,Tied to charger for conversations lasting more...,negative
4,The mic is great.,positive


In [3]:
df1['Cleaned_Text'] = df1['sentence'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df2['Cleaned_Text']=df2['sentence'].str.replace(r'[^a-zA-Z0-9\s]','',regex=True)

df1['Cleaned_Text'] = df1['Cleaned_Text'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
df2['Cleaned_Text']=df2['Cleaned_Text'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()

In [4]:
print(f'df1',df1.shape)
print(f'df2',df2.shape)

df1 (422746, 3)
df2 (3309, 3)


In [5]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.0 MB/s eta 0:00:00:00:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
df1.head()

,sentence,emotion,Cleaned_Text
0,i just feel really helpless and heavy hearted,fear,i just feel really helpless and heavy hearted
1,ive enjoyed being able to slouch about relax a...,sad,ive enjoyed being able to slouch about relax a...
2,i gave up my internship with the dmrg and am f...,fear,i gave up my internship with the dmrg and am f...
3,i dont know i feel so lost,sad,i dont know i feel so lost
4,i am a kindergarten teacher and i am thoroughl...,fear,i am a kindergarten teacher and i am thoroughl...


In [7]:
from tqdm import tqdm
import spacy

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
tqdm.pandas()

# Function for batch lemmatization
def spacy_batch_lemmatizer(texts):
    docs = nlp.pipe(texts, batch_size=1000)
    return [' '.join([token.lemma_ for token in doc]) for doc in tqdm(docs, total=len(texts))]

# Apply to your cleaned text column
df1['Lemmatized_Text'] = spacy_batch_lemmatizer(df1['Cleaned_Text'])


100%|██████████| 422746/422746 [09:47<00:00, 719.75it/s] 


In [8]:
df2['Lemmatized_Text'] = spacy_batch_lemmatizer(df2['Cleaned_Text'])


100%|██████████| 3309/3309 [00:03<00:00, 1011.58it/s]


In [9]:
df2.head()

,sentence,sentiment,Cleaned_Text,Lemmatized_Text
0,So there is no way for me to plug it in here i...,negative,so there is no way for me to plug it in here i...,so there be no way for I to plug it in here in...
1,"Good case, Excellent value.",positive,good case excellent value,good case excellent value
2,Great for the jawbone.,positive,great for the jawbone,great for the jawbone
3,Tied to charger for conversations lasting more...,negative,tied to charger for conversations lasting more...,tie to charger for conversation last more than...
4,The mic is great.,positive,the mic is great,the mic be great


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts1 = df1['Lemmatized_Text']
vectorizer1 = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), stop_words='english')
X1_tfidf = vectorizer1.fit_transform(texts1)

texts2 = df2['Lemmatized_Text']
vectorizer2 = TfidfVectorizer(max_features=10000, ngram_range=(1, 2), stop_words='english')
X2_tfidf = vectorizer2.fit_transform(texts2)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1_tfidf, df1['emotion'], test_size=0.2, random_state=42)

# Train model
model1 = MultinomialNB()
model1.fit(X_train1, y_train1)

# Evaluate
y_pred1 = model1.predict(X_test1)
print("Emotion Classification Accuracy:", accuracy_score(y_test1, y_pred1))
print(classification_report(y_test1, y_pred1))


Emotion Classification Accuracy: 0.8731519810762862
              precision    recall  f1-score   support

       anger       0.94      0.85      0.89     11810
        fear       0.88      0.81      0.85      9952
         joy       0.84      0.96      0.89     28781
        love       0.87      0.58      0.70      6929
         sad       0.89      0.95      0.92     24036
     suprise       0.88      0.48      0.63      3042

    accuracy                           0.87     84550
   macro avg       0.88      0.77      0.81     84550
weighted avg       0.88      0.87      0.87     84550



In [12]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2_tfidf, df2['sentiment'], test_size=0.2, random_state=42)

model2 = MultinomialNB()
model2.fit(X_train2, y_train2)

# Evaluate
y_pred2 = model2.predict(X_test2)
print("Sentiment Classification Accuracy:", accuracy_score(y_test2, y_pred2))
print(classification_report(y_test2, y_pred2))


Sentiment Classification Accuracy: 0.8006042296072508
              precision    recall  f1-score   support

    negative       0.83      0.77      0.80       334
    positive       0.78      0.84      0.81       328

    accuracy                           0.80       662
   macro avg       0.80      0.80      0.80       662
weighted avg       0.80      0.80      0.80       662



In [13]:
df1.to_csv('combined_emotion_cleaned.csv', index=False)

df2.to_csv('combined_sentiment_cleaned.csv', index=False)


In [14]:

results_df1 = pd.DataFrame({
    'Actual': y_test1.values,
    'Predicted': y_pred1
})

results_df1['Text'] = df1.loc[y_test1.index, 'Lemmatized_Text'].values

results_df1.to_csv('emotion_test_predictions.csv', index=False)


In [15]:
results_df2 = pd.DataFrame({
    'Actual': y_test2.values,
    'Predicted': y_pred2
})

# Optionally include the original text
results_df2['Text'] = df2.loc[y_test2.index, 'Lemmatized_Text'].values

# Save to CSV
results_df2.to_csv('sentiment_test_predictions.csv', index=False)
